#### Analyzing resume by making a QnA pipeline using Langchain, using concept of RAG by inputting a PDF a.k.a data source and querying this using vector store also

In [2]:
# Imports

from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain


##### Taking a sample resume and then analyzing that

In [4]:
pdf_path="C:\\Users\\91868\\Downloads\\J2EE Developer.pdf"
pdf_reader=PdfReader(pdf_path)
pdf_reader

In [5]:
# Extracting text from each page
text=""
for page in pdf_reader.pages:
    text+=page.extract_text()
print(text)

ABOUT
I am an experienced software developer, seeking a full-time position in the
field of  backend development where I can apply my knowledge and skills for
continuous improvement.
WORK EXPERIENCE
EDUCATION
BACHELOR OF TECHNOLOGY (HONS.), MAJOR IN ELECTRONICS AND COMM.
AKTU | 2016-2020
Percentage 83.46%
AWARDS & CERTIFICATIONS
Best Newcomer & On the Spot Awards
TCS | OCT 2021
For outstanding quality of work, customer appreciation, resolving and turning
around escalations and significant contribution towards project
Certified SAFe 5 (Scaled Agile Framework for Enterprise) Practitioner with ID:
17588 169-4782SOFTWARE ENGINEER 
Tata Consultancy Services (TCS) | JAN 2021 - Present
Project: TravelPort (Travel Domain Services)
Handling responsibilities of development, bug fixes, requirement analysis,
effort estimation.
Developing Microservices for Airlines like (BA, LH, SQ, AA, UA, SN, QF) for
E2E ticket booking system
Involved in enhancing the existing Microservices functionality and fixin

In [6]:
# Splitting long text into chunks to fit context limit
text_split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100,length_function=len)
chunks=text_split.split_text(text=text)
chunks

['ABOUT\nI am an experienced software developer, seeking a full-time position in the\nfield of  backend development where I can apply my knowledge and skills for\ncontinuous improvement.\nWORK EXPERIENCE\nEDUCATION\nBACHELOR OF TECHNOLOGY (HONS.), MAJOR IN ELECTRONICS AND COMM.\nAKTU | 2016-2020\nPercentage 83.46%\nAWARDS & CERTIFICATIONS\nBest Newcomer & On the Spot Awards\nTCS | OCT 2021\nFor outstanding quality of work, customer appreciation, resolving and turning',
 'TCS | OCT 2021\nFor outstanding quality of work, customer appreciation, resolving and turning\naround escalations and significant contribution towards project\nCertified SAFe 5 (Scaled Agile Framework for Enterprise) Practitioner with ID:\n17588 169-4782SOFTWARE ENGINEER \nTata Consultancy Services (TCS) | JAN 2021 - Present\nProject: TravelPort (Travel Domain Services)\nHandling responsibilities of development, bug fixes, requirement analysis,\neffort estimation.',
 'Handling responsibilities of development, bug fixes

In [7]:
len(chunks)

7

In [8]:
chunks[4]

'proper functionality of deployed Micro Services\nDeploying services in Jenkins and configuring new Microservices on\nZookeeper. \nKnowledge of Scale Agile framework and cross-functional teams\nTechnology: Java 8, Spring Boot, Hibernate, JPA\nData Base: MSSQL Server\nFramework: Spring, Spring Boot, Hibernate, JPA\nEnvironment: Windows\nCONTACT\nP: +91-9795864320\nA: Gurugram, Delhi-NCR\nE: prishabh214@gmail.com\nLi: https://www.linkedin.com/in/rishabh-         \npandey-b0a459153/Methodology: Agile, Lean'

In [9]:
def openai(chunks,query_):
    # use openai embeddings
    embeddings=OpenAIEmbeddings()
    # FAISS to convert text data to numerical data
    vectorstores=FAISS.from_texts(chunks,embedding=embeddings)
    # compare query & chunks,select top k most similar chunks based on similarity scores
    docs=vectorstores.similarity_search(query=query_,k=3)
    # creating openai object
    llm=ChatOpenAI(model="gpt-3.5-turbo")
    # QA pipeline using chain function
    chain=load_qa_chain(llm=llm,chain_type='stuff')
    # running the chain
    response=chain.run(input_documents=docs,question=query_)
    return response

In [10]:
# Writing fucntion to provide summary of the resume
def return_resume_summary(chunked_query):
    query=f''' need to provide detailed summarization of below resume and finally conclude them
               
                """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {chunked_query}

                """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
            '''
    return query

# testing this method
summary=return_resume_summary(chunked_query=chunks)
summary_result=openai(chunks=chunks,query_=summary)
print(summary_result)

Based on the provided resume details, here is a detailed summarization:

1. **Education:** Completed Bachelor of Technology with a major in Electronics and Communication from AKTU in 2020 with a percentage of 83.46%.

2. **Work Experience:** Currently working as a Software Engineer at Tata Consultancy Services (TCS) since January 2021. Working on the TravelPort project in the Travel Domain Services. Responsibilities include development, bug fixes, requirement analysis, and effort estimation. Developing Microservices for airlines like BA, LH, SQ, AA, UA, SN, QF for an end-to-end ticket booking system.

3. **Technical Skills:** Proficient in Java 8, Spring Boot, Hibernate, JPA, J2EE, Spring MVC, multi-threading, collections, streams, and REST controllers. Experienced with MSSQL Server, Git, GitAction, Tomcat, Maven, and various tools like Postman, SOAP UI, Jenkins, Jira, Confluence, etc.

4. **Certifications & Awards:** Certified SAFe 5 (Scaled Agile Framework for Enterprise) Practitione

In [13]:
# MEthod to return strength of the candidate
def return_resume_strength(chunked_query):
    query=f'''need to provide detailed analysis and explain of the strength of below resume and finally conclude them
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {chunked_query}
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

strength=return_resume_strength(chunked_query=summary_result)
strength_result=openai( chunks=chunks, query_=strength)
print(strength_result)

The strength of the provided resume lies in the candidate's comprehensive education background, current work experience at Tata Consultancy Services (TCS) with a focus on backend development and microservices, proficiency in Java and related technologies, as well as the certifications obtained, such as being a Certified SAFe 5 Practitioner. Additionally, the candidate's professional skills, including creativity, reliability, and teamwork, contribute positively to their profile.

The resume showcases a strong foundation in both theoretical knowledge and practical experience, making the candidate a well-rounded software developer suitable for roles requiring backend development expertise. The combination of technical expertise, project experience, certifications, and professional skills makes the candidate a competitive choice for positions in the field of backend development.


In [14]:
# Method to return weakness of cancidate
def return_resume_weakness(chunked_query):
    query=f'''need to detailed analysis and explain of the weakness of below resume and how to improve make a better resume.

                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                {chunked_query}
                """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

weakness=return_resume_weakness(chunked_query=summary_result)
result_weakness=openai(chunks=chunks, query_=weakness)
print(result_weakness)

Based on the provided resume, here are some areas that can be improved for a better resume:

1. **Weaknesses:**
   - The resume lacks a clear objective or summary statement at the beginning. Adding a concise summary highlighting the candidate's primary expertise and career goals can make the resume more impactful.
   - The work experience section could benefit from more quantifiable achievements or projects completed. Adding specific examples of successful projects, impact on the business, or efficiencies improved can enhance the credibility of the experience.
   - While the technical skills section is comprehensive, it might be overwhelming to read as a block of text. Consider organizing it into subcategories or highlighting key technologies that are most relevant to the target position.
   - The resume could include information about any extracurricular activities, volunteer work, or relevant hobbies that demonstrate additional skills or interests. This can add a more well-rounded pr

In [16]:
# Method to return job titles suggestions
def return_job_title_suggestion(chunked_query):

    query = f''' what are the job roles the candidate can apply to likedin based on below,along with expected pay-scale in indian rupees?
                  
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                  {chunked_query}
                  """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
                '''
    return query

suggestion=return_job_title_suggestion(chunked_query=summary_result)
result_suggestion=openai(chunks=chunks, query_=suggestion)
print(result_suggestion)

Based on the resume details provided, the candidate can apply for job roles such as:
1. Backend Developer
2. Software Engineer
3. Java Developer
4. Microservices Developer
5. Full Stack Developer

Expected pay-scale in Indian Rupees may vary based on factors like location, company size, and experience level. However, for a candidate with their qualifications and experience, a potential pay scale could range from ₹6-12 lakhs per annum for entry-level positions to ₹12-20 lakhs per annum for mid-level roles in the mentioned job profiles. For senior positions, the pay scale could go higher. It's important to note that these are general estimates and actual salaries may vary.
